In [1]:
import numpy as np
import pandas as pd

In [2]:
#Prevent Large File crashing on Jupyter Notebook
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Filter out the asins that aren't in Goodreads

### Work on amazonData_1.csv

In [3]:
asin = pd.read_csv('bookId_vs_asin.csv')

In [4]:
amazon1 = pd.read_csv('amazonData_1.csv')

In [5]:
amazon1 = pd.merge(amazon1,asin, on='asin', how='inner')

In [6]:
amazon1

,overall,reviewTime,asin,reviewText,book_id
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur...",446759
1,5.0,"06 20, 2016",0001713353,The kids loved it!,446759
2,5.0,"01 24, 2016",0001713353,My students (3 & 4 year olds) loved this book!...,446759
3,5.0,"07 9, 2015",0001713353,LOVE IT,446759
4,5.0,"01 18, 2015",0001713353,Great!,446759
...,...,...,...,...,...
1555844,5.0,"01 13, 2007",0140106995,A wonderful rampage through the twentieth cent...,880185
1555845,4.0,"09 18, 2005",0140106995,William Boyd is a terrific storyteller. His p...,880185
1555846,4.0,"09 18, 2005",0140106995,William Boyd is a terrific storyteller. His p...,880185
1555847,5.0,"12 24, 2003",0140106995,This fictional memoir displays Boyd's consumma...,880185


In [7]:
amazon1.to_csv('official_amazon.csv',index=False)

### Work on amazonData_2.csv

In [8]:
amazon2 = pd.read_csv('amazonData_2.csv')

In [9]:
amazon2 = pd.merge(amazon2,asin, on='asin', how='inner')

In [10]:
amazon2

,overall,reviewTime,asin,reviewText,book_id
0,4.0,"07 31, 2003",0140106995,I'm a sucker for books that chronicle a man's ...,880185
1,4.0,"07 31, 2003",0140106995,I'm a sucker for books that chronicle a man's ...,880185
2,4.0,"11 28, 1999",0140140360,"These essays are interesting, if only because ...",537762
3,5.0,"05 13, 1998",0140152601,Van Doren's biography of Franklin is nothing s...,1558473
4,5.0,"12 4, 2013",0140152601,I really do love this book. It was passed on ...,1558473
...,...,...,...,...,...
1641625,3.0,"06 6, 2008",0312947763,THIS WAS JUST AN OK TRUE CRIME BOOK. I WISH TH...,2830423
1641626,1.0,"03 12, 2009",0312947763,I haven't much more to add to review of this b...,2830423
1641627,2.0,"10 25, 2008",0312947763,Michele R. McPhee writes in a style that is no...,2830423
1641628,1.0,"07 25, 2008",0312947763,This was not a sarisfying crime book. The back...,2830423


In [11]:
with open('official_amazon.csv', 'a') as f:
    amazon2.to_csv(f, header=False, index = False)

### Check duplicated reviews

In [3]:
amazon = pd.read_csv('official_amazon.csv')

In [4]:
amazon

,overall,reviewTime,asin,reviewText,book_id
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur...",446759
1,5.0,"06 20, 2016",0001713353,The kids loved it!,446759
2,5.0,"01 24, 2016",0001713353,My students (3 & 4 year olds) loved this book!...,446759
3,5.0,"07 9, 2015",0001713353,LOVE IT,446759
4,5.0,"01 18, 2015",0001713353,Great!,446759
...,...,...,...,...,...
3197474,3.0,"06 6, 2008",0312947763,THIS WAS JUST AN OK TRUE CRIME BOOK. I WISH TH...,2830423
3197475,1.0,"03 12, 2009",0312947763,I haven't much more to add to review of this b...,2830423
3197476,2.0,"10 25, 2008",0312947763,Michele R. McPhee writes in a style that is no...,2830423
3197477,1.0,"07 25, 2008",0312947763,This was not a sarisfying crime book. The back...,2830423


In [5]:
# Get duplicated reviews
amazon[amazon[['asin', 'overall','reviewTime','reviewText']].duplicated() == True]

,overall,reviewTime,asin,reviewText,book_id
665,5.0,"02 1, 2014",0002051850,Ernest Hemingway's FOR WHOM THE BELL TOLLS is ...,9924403
1356,5.0,"09 11, 2004",000171287X,The text consists of words beginning with B wi...,260996
2131,5.0,"04 21, 2004",0001713256,This was one of my favorites as a child. It b...,90843
2483,5.0,"10 13, 2015",000100039X,"I read this book not knowing what to expect, a...",2547
2484,5.0,"10 13, 2015",000100039X,"I read this book not knowing what to expect, a...",2547
...,...,...,...,...,...
3189808,5.0,"12 29, 2013",0312945795,Have read the series and recommend this books ...,2721318
3189810,5.0,"12 29, 2013",0312945795,Lora Leigh continues her breed series in this ...,2721318
3193067,5.0,"11 10, 2014",031294232X,Loved,6434232
3194130,5.0,"09 24, 2014",0312934106,just as agreed,362685


In [12]:
#Checking
pd.options.display.max_rows = 100
amazon[(amazon.asin =='0312944632')  & (amazon.overall == 5.0) & (amazon.reviewTime =='03 7, 2014') ]

,overall,reviewTime,asin,reviewText,book_id
3194171,5.0,"03 7, 2014",0312944632,Lots of great info and insights into the makin...,77102
3194172,5.0,"03 7, 2014",0312944632,Lots of great info and insights into the makin...,77102


In [13]:
# Drop duplicated reviews
ama = amazon.drop_duplicates(subset=['asin', 'overall','reviewTime','reviewText'], keep= 'first')

In [14]:
ama

,overall,reviewTime,asin,reviewText,book_id
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur...",446759
1,5.0,"06 20, 2016",0001713353,The kids loved it!,446759
2,5.0,"01 24, 2016",0001713353,My students (3 & 4 year olds) loved this book!...,446759
3,5.0,"07 9, 2015",0001713353,LOVE IT,446759
4,5.0,"01 18, 2015",0001713353,Great!,446759
...,...,...,...,...,...
3197474,3.0,"06 6, 2008",0312947763,THIS WAS JUST AN OK TRUE CRIME BOOK. I WISH TH...,2830423
3197475,1.0,"03 12, 2009",0312947763,I haven't much more to add to review of this b...,2830423
3197476,2.0,"10 25, 2008",0312947763,Michele R. McPhee writes in a style that is no...,2830423
3197477,1.0,"07 25, 2008",0312947763,This was not a sarisfying crime book. The back...,2830423


In [15]:
ama.to_csv('official_Amazon.csv',index = False)